## previously we have flatten the embeddigns
## now we only test with the converted embedding and same result (tiny slight improve)
- tried with imbalance data,  tfidf (best features 6300) ---- not worked
- tried with imbalance data, wcbtfidf ( best features 1500) ---- not worked

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import joblib
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split # fnding max_features
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
# import joblib
import os

# hyperparameter tuning
from sklearn.ensemble import RandomForestRegressor # to look availiable hyperparameters
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # 

import ast


In [2]:
df = pd.read_csv('created_new_train_data/new_11.csv') #--- we creaed this dataset because, we have lost the first ipy file, thats why regenerated the csv

In [7]:
# df.columns

In [8]:
# df.head()

In [138]:
#df = pd.read_csv('new_10.csv') # test data

In [3]:
df.head()# test data

,Unnamed: 0,Sentence_id,Text,class_label,Tokens,text_length,sentiment_score_veda,sentiment_score_textblob,sentiment_score_bert,sentiment_score_roberta,...,cleaned_text_length,contains_question_mark,contains_exclamation,contains_ellipsis,num_exclamations,num_questions,num_ellipses,punctuation_count,converted_embedding,text-tfidf
0,0,30313,And so I know that this campaign has caused so...,No,"['campaign', 'caused', 'questioning', 'worries...",118,-0.4939,0.5,"[0.9687058329582214, 0.031294114887714386]","[0.11130036413669586, 0.014399373903870583, 0....",...,117,False,False,False,0,0,0,1,[-2.65752003e-02 2.48833038e-02 -1.60991415e-...,campaign cause question worry part leader acro...
1,1,19099,"Now, let's balance the budget and protect Medi...",No,"['lets', 'balance', 'budget', 'protect', 'medi...",92,0.3818,0.0,"[0.5200970768928528, 0.4799029529094696]","[0.10983521491289139, 0.03143635019659996, 0.7...",...,87,False,False,False,0,0,0,5,[-6.90975189e-02 -5.70267588e-02 -1.18759215e-...,let balance budget protect medicare medicaid e...
2,2,33964,I'd like to mention one thing.,No,"['id', 'mention', 'thing']",30,0.3612,0.0,"[0.22529488801956177, 0.774705171585083]","[0.04292938485741615, 0.5959416627883911, 0.27...",...,28,False,False,False,0,0,0,2,[ 1.57115221e-01 3.26707251e-02 -3.22537839e-...,I d mention thing
3,3,16871,I must remind him the Democrats have controlle...,Yes,"['must', 'remind', 'democrats', 'controlled', ...",124,0.0000,0.0,"[0.9874435067176819, 0.012556521221995354]","[0.31787270307540894, 0.20612367987632751, 0.3...",...,122,False,False,False,0,0,0,2,[-9.69030559e-02 4.20815766e-01 -1.68550998e-...,must remind democrats control congress last tw...
4,4,13150,And to take a chance uh - now be - and not mak...,No,"['take', 'chance', 'effort', 'provide', 'contr...",161,0.2023,0.8,"[0.999810516834259, 0.00018953572725877166]","[0.5015895366668701, 0.01270473375916481, 0.27...",...,155,False,False,False,0,0,0,4,[ 1.04969844e-01 3.35686594e-01 -3.11189741e-...,take chance effort provide control weapon grea...


In [7]:
df.rename(columns={'converted_embedding':'embedding'}, inplace = True)

In [8]:
df.dtypes

Unnamed: 0                      int64
Sentence_id                     int64
Text                           object
class_label                    object
Tokens                         object
text_length                     int64
sentiment_score_veda          float64
sentiment_score_textblob      float64
sentiment_score_bert           object
sentiment_score_roberta        object
bert_sent_neg                 float64
bert_sent_pos                 float64
roberta_sent_neg              float64
roberta_sent_neut             float64
roberta_sent_mixed            float64
roberta_sent_pos              float64
labels                          int64
names                           int64
organizations                   int64
locations                       int64
dates                           int64
verbs                          object
action_verbs                   object
filtered_action_verbs          object
joined_tokens                  object
count_verb                      int64
count_action

In [9]:
# convert the embeddnigs into numpy array and then flatten the array and store

def convert_to_numpy_array(embd_str):
    #print(f"string: {embd_str}")
    #print(type(embd_str))
    embd_str_clean = embd_str.replace("\n", " ").replace(" ", " ")
    #print(type(embd_str_clean))
    #print(f"string cleansing: {embd_str_clean}")
    embd_str_clean = embd_str_clean.strip('[]')
    #print(type(embd_str_clean))
    #print(f"removing the outer brackets {embd_str_clean}")
    embd_list = embd_str_clean.split()
    #print(f"Split the string into individual values {embd_list}")
    embed_array = np.array(embd_list, dtype = float)
    #print(f"list to array {embed_array}")
    return embed_array

In [10]:
df['converted_embedding'] = df['converted_embedding'].apply(convert_to_numpy_array)

In [20]:
df['converted_embedding'].iloc[0]

array([-2.65752003e-02,  2.48833038e-02, -1.60991415e-01, -3.71651500e-01,
       -3.86815250e-01, -5.26572466e-01, -3.39005254e-02,  5.57528257e-01,
        5.22308238e-03, -5.07022440e-01,  2.28680804e-01, -4.05779667e-03,
        1.02525599e-01,  4.42762583e-01,  3.72143209e-01,  2.71828473e-01,
       -2.42723987e-01,  5.83807111e-01,  1.45760924e-01, -1.44843876e-01,
        1.63174480e-01, -6.85929775e-01,  4.87132072e-01,  2.95782268e-01,
        2.69547790e-01, -1.93092138e-01, -3.34469788e-02, -2.56053388e-01,
       -3.86430472e-01,  1.24803938e-01, -1.97812960e-01,  4.92808260e-02,
       -5.94320118e-01,  4.87280264e-02,  3.27084094e-01, -2.57128552e-02,
       -1.27284572e-01,  1.57622397e-01,  9.59223434e-02,  3.40999514e-02,
       -2.21919730e-01,  1.39919311e-01,  8.71447027e-02, -1.46107078e-02,
        1.96743011e-02, -5.71499348e-01, -3.48456144e+00,  2.87487283e-02,
        1.11168951e-01,  1.95358768e-02, -1.61416158e-01,  2.23957617e-02,
        9.47843194e-02,  

In [13]:
df.to_csv('test_embedding.csv', index = False)

In [14]:
new_df = pd.read_csv('test_embedding.csv')

In [18]:
new_df['converted_embedding'].iloc[0]

'[-2.65752003e-02  2.48833038e-02 -1.60991415e-01 -3.71651500e-01\n -3.86815250e-01 -5.26572466e-01 -3.39005254e-02  5.57528257e-01\n  5.22308238e-03 -5.07022440e-01  2.28680804e-01 -4.05779667e-03\n  1.02525599e-01  4.42762583e-01  3.72143209e-01  2.71828473e-01\n -2.42723987e-01  5.83807111e-01  1.45760924e-01 -1.44843876e-01\n  1.63174480e-01 -6.85929775e-01  4.87132072e-01  2.95782268e-01\n  2.69547790e-01 -1.93092138e-01 -3.34469788e-02 -2.56053388e-01\n -3.86430472e-01  1.24803938e-01 -1.97812960e-01  4.92808260e-02\n -5.94320118e-01  4.87280264e-02  3.27084094e-01 -2.57128552e-02\n -1.27284572e-01  1.57622397e-01  9.59223434e-02  3.40999514e-02\n -2.21919730e-01  1.39919311e-01  8.71447027e-02 -1.46107078e-02\n  1.96743011e-02 -5.71499348e-01 -3.48456144e+00  2.87487283e-02\n  1.11168951e-01  1.95358768e-02 -1.61416158e-01  2.23957617e-02\n  9.47843194e-02  1.62391469e-01  3.77150267e-01  5.52754775e-02\n -5.82214475e-01  3.88859719e-01 -4.52454001e-01 -4.95950758e-01\n  3.21322

In [1]:
# pca = PCA(n_components = 356)
# reduced_embeddings = pca.fit_transform(df['converted_embedding'].to_list())

In [71]:
# df.to_csv('new_9.csv', index = False) # creating new csv with converted_embedidng column in test data set

# trying to test the existing model without flatten the converted_embedding

In [26]:
# gold  test dataset
gold_df = pd.read_csv('english_test_08_01.csv')
gold_df.head()
# use converted embedding in the pca
# test the model

,Sentence_id,Text,Tokens,text_length,cleaned_text,cleaned_text_length,roberta_sentiment,roberta_sent_pos,roberta_sent_neg,roberta_sent_mixed,roberta_sentiment_class,count_tokens,counts,names,organizations,dates,punctuation_count,embedding,converted_embedding,flattened_embedding
0,35988,They said they were just going to get inspecto...,"['said', 'going', 'get', 'inspectors']",52,said going get inspectors,25,"{'positive_score': 0.03105599619448185, 'negat...",0.031056,0.248589,0.720355,Mixed,4,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0,0,[[ 3.47921252e-01 1.69657081e-01 2.36351147e...,[[ 3.47921252e-01 1.69657081e-01 2.36351147e...,[ 3.47921252e-01 1.69657081e-01 2.36351147e-...
1,35991,"And from that point on, I've voted to -- I mov...","['point', 'Ive', 'voted', 'moved', 'bring', 't...",76,point ive voted moved bring troops home,39,"{'positive_score': 0.08640998601913452, 'negat...",0.086410,0.079362,0.834228,Mixed,7,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0,0,[[-6.57255352e-02 1.03908114e-01 -1.68640435e...,[[-6.57255352e-02 1.03908114e-01 -1.68640435e...,[-6.57255352e-02 1.03908114e-01 -1.68640435e-...
2,36029,I sit on the Senate Armed Services Committee.,"['sit', 'Senate', 'Armed', 'Services', 'Commit...",45,sit senate armed services committee,35,"{'positive_score': 0.020918136462569237, 'nega...",0.020918,0.045755,0.933327,Mixed,5,"{'names': 0, 'organizations': 1, 'dates': 0}",0,1,0,0,[[-4.35761958e-01 -3.96292239e-01 -1.37289584e...,[[-4.35761958e-01 -3.96292239e-01 -1.37289584e...,[-4.35761958e-01 -3.96292239e-01 -1.37289584e-...
3,36043,We need to depend on all of our tools -- diplo...,"['need', 'depend', 'tools', 'diplomatic', 'eco...",138,need depend tools diplomatic economic working ...,84,"{'positive_score': 0.11722128093242645, 'negat...",0.117221,0.164593,0.718186,Mixed,12,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0,0,[[-2.99328677e-02 3.05866241e-01 -3.39354366e...,[[-2.99328677e-02 3.05866241e-01 -3.39354366e...,[-2.99328677e-02 3.05866241e-01 -3.39354366e-...
4,36063,And that is -- and I don't know if my colleagu...,"['dont', 'know', 'colleagues', 'agree']",80,dont know colleagues agree,26,"{'positive_score': 0.02025897055864334, 'negat...",0.020259,0.672274,0.307467,Negative,4,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0,0,[[ 9.60470885e-02 3.00483525e-01 -8.92113447e...,[[ 9.60470885e-02 3.00483525e-01 -8.92113447e...,[ 9.60470885e-02 3.00483525e-01 -8.92113447e-...


In [31]:
gold_df['new_converted_embedding'] = gold_df['embedding'].apply(convert_to_numpy_array)

In [35]:
# gold_df['new_converted_embedding'].iloc[0]


In [34]:
# drop the converted_embedding and flattened embedding column
gold_df.drop(['converted_embedding', 'flattened_embedding'], axis = 1, inplace = True)

In [36]:
gold_df.head(2)

,Sentence_id,Text,Tokens,text_length,cleaned_text,cleaned_text_length,roberta_sentiment,roberta_sent_pos,roberta_sent_neg,roberta_sent_mixed,roberta_sentiment_class,count_tokens,counts,names,organizations,dates,punctuation_count,embedding,new_converted_embedding
0,35988,They said they were just going to get inspecto...,"['said', 'going', 'get', 'inspectors']",52,said going get inspectors,25,"{'positive_score': 0.03105599619448185, 'negat...",0.031056,0.248589,0.720355,Mixed,4,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0,0,[[ 3.47921252e-01 1.69657081e-01 2.36351147e...,"[0.347921252, 0.169657081, 0.236351147, -0.127..."
1,35991,"And from that point on, I've voted to -- I mov...","['point', 'Ive', 'voted', 'moved', 'bring', 't...",76,point ive voted moved bring troops home,39,"{'positive_score': 0.08640998601913452, 'negat...",0.086410,0.079362,0.834228,Mixed,7,"{'names': 0, 'organizations': 0, 'dates': 0}",0,0,0,0,[[-6.57255352e-02 1.03908114e-01 -1.68640435e...,"[-0.0657255352, 0.103908114, -0.168640435, -0...."


In [39]:
embeddings = np.vstack(gold_df['new_converted_embedding']) # generate columns

In [48]:
pca = PCA(n_components = 50)
reduced_embeddings = pca.fit_transform(gold_df['new_converted_embedding'].to_list())
reduced_embeddings = np.array(reduced_embeddings, dtype = np.float32)

In [41]:
additional_features = gold_df[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [49]:
X = np.hstack([reduced_embeddings, additional_features]) # stack horizontally to add additional features with corresponding embedding vectors
# y = gold_df['labels']

In [50]:
model_path_lr = 'models/logistic_reggression.pkl'
model_lr = joblib.load(model_path_lr)
predictions_lr = model_lr.predict(X)
predictions_lr

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [51]:
model_path_rfc = 'models/rfc.pkl'
model_rfc = joblib.load(model_path_rfc)
predictions_rfc = model_lr.predict(X)
predictions_rfc

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [53]:
test_data_lr = pd.DataFrame({
    'id': gold_df['Sentence_id'],
    'predictions': predictions_lr
})
test_data_lr['predictions'] = test_data_lr['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
print(test_data_lr[test_data_lr['predictions'] == 'Yes'].count())
print(test_data_lr[test_data_lr['predictions'] == 'No'].count())

id             21
predictions    21
dtype: int64
id             320
predictions    320
dtype: int64


In [55]:
test_data_rfc = pd.DataFrame({
    'id': gold_df['Sentence_id'],
    'predictions': predictions_rfc
})
test_data_rfc['predictions'] = test_data_rfc['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
print(test_data_rfc[test_data_rfc['predictions'] == 'Yes'].count())
print(test_data_rfc[test_data_rfc['predictions'] == 'No'].count())

id             21
predictions    21
dtype: int64
id             320
predictions    320
dtype: int64


In [56]:
real_gold_df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/test_english_gold/CT24_checkworthy_english_test_gold.tsv',delimiter = '\t')

In [60]:
## for LR
df_merged_lr = pd.merge(real_gold_df, test_data_lr, left_on='Sentence_id', right_on='id', how='inner')

# Correct the typo: 'gold_lables' to 'gold_labels'
gold_labels = df_merged_lr['class_label'].tolist()
pred_labels = df_merged_lr['predictions'].tolist()

# Now use 'gold_labels' consistently
acc = accuracy_score(gold_labels, pred_labels)
precision = precision_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
recall = recall_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels, pred_labels, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7390029325513197
Precision: 0.47619047619047616
Recall: 0.11363636363636363
F1 Score: 0.1834862385321101


In [61]:
# for rfc
df_merged_rfc = pd.merge(real_gold_df, test_data_rfc, left_on='Sentence_id', right_on='id', how='inner')

# Correct the typo: 'gold_lables' to 'gold_labels'
gold_labels = df_merged_rfc['class_label'].tolist()
pred_labels = df_merged_rfc['predictions'].tolist()

# Now use 'gold_labels' consistently
acc = accuracy_score(gold_labels, pred_labels)
precision = precision_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
recall = recall_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels, pred_labels, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7390029325513197
Precision: 0.47619047619047616
Recall: 0.11363636363636363
F1 Score: 0.1834862385321101


### Model Performance Metrics LINEAR REGRESSION
### 1_english_test group1 and checkworthy_3 group2

| Metric      | Group 1 Score (flatten embedding)   | Group 2 Score (converted to numpy)   |
|-------------|-------------------------------------|--------------------------------------|
| Accuracy    | 0.7361                              | 0.7390                               |
| Precision   | 0.4444                              | 0.4762                               |
| Recall      | 0.0909                              | 0.1136                               |
| F1 Score    | 0.1509                              | 0.1835                               |


### Model Performance Metrics RANDOM FOREST CLASSIFIER
### 1_english_test group1 and checkworthy_3 group2

| Metric      | Group 1 Score (flatten embedding)   | Group 2 Score (converted to numpy)   |
|-------------|-------------------------------------|--------------------------------------|
| Accuracy    | 0.7361                              | 0.7390                               |
| Precision   | 0.4444                              | 0.4762                               |
| Recall      | 0.0909                              | 0.1136                               |
| F1 Score    | 0.1509                              | 0.1835                               |


###  <span style="color:blue;"> Try without redusing dimensionality </span>
### <span style="color:red;"> Is there any option to check the best n_component in PCA ? not checked (later)</span>


In [3]:
new_df = pd.read_csv('created_new_train_data/new_11.csv')

In [141]:
# new_df.drop('embedding', axis = 1,inplace = True)

In [157]:
new_df.shape

(22501, 40)

In [7]:
# new_df.to_csv('new_10.csv', index = False)

In [4]:
new_df.columns

Index(['Unnamed: 0', 'Sentence_id', 'Text', 'class_label', 'Tokens',
       'text_length', 'sentiment_score_veda', 'sentiment_score_textblob',
       'sentiment_score_bert', 'sentiment_score_roberta', 'bert_sent_neg',
       'bert_sent_pos', 'roberta_sent_neg', 'roberta_sent_neut',
       'roberta_sent_mixed', 'roberta_sent_pos', 'labels', 'names',
       'organizations', 'locations', 'dates', 'verbs', 'action_verbs',
       'filtered_action_verbs', 'joined_tokens', 'count_verb',
       'count_action_verb', 'count_filtered_action_verb', 'count_tokens',
       'cleaned_text', 'cleaned_text_length', 'contains_question_mark',
       'contains_exclamation', 'contains_ellipsis', 'num_exclamations',
       'num_questions', 'num_ellipses', 'punctuation_count',
       'converted_embedding', 'text-tfidf'],
      dtype='object')

In [12]:
new_df['joined_tokens'].iloc[0]

'campaign caused questioning worries part leaders across globe'

In [13]:
new_df['Text'].iloc[0]

'And so I know that this campaign has caused some questioning and worries on the part of many leaders across the globe.'

In [14]:
new_df['Tokens'].iloc[0]

"['campaign', 'caused', 'questioning', 'worries', 'part', 'leaders', 'across', 'globe']"

new_df.describe()

In [17]:
desc_output = new_df.describe()

In [18]:
describe_df = desc_output.reset_index()

In [20]:
describe_df.to_csv('decribe.csv')

# TF IDF

In [6]:
text = new_df['Text']
tfidf_vect = TfidfVectorizer(lowercase = True, analyzer='word', stop_words = 'english', ngram_range=(1, 2))
tfidf_features = tfidf_vect.fit(text)

In [7]:
tfidf_features.vocabulary_.__len__()

110838

In [45]:
# print(new_df['Text'].iloc[3])

In [48]:
# tfidf_features.vocabulary_
new_df['joined_tokens'].iloc[3]

'must remind democrats controlled congress last twentytwo wrote bills'

## after generating tokens, there is some column with null value in joined tokens, just dropppd it, because wcb-tfidf can not handle

In [169]:
new_df = new_df.dropna(subset=['joined_tokens'])

In [170]:
new_df.shape

(22385, 40)

##  finding best tf-idf max_feature

## new modification - 11/2
- create new_column from joined_tokens -> lemmatize.
- train a tfidf model on new best features.
- save as  new tfidf-model.

In [147]:
import spacy

In [148]:
nlp = spacy.load('en_core_web_sm')
def lemmatiz_text(text):
    doc = nlp(text)
    lemma_tokens = [token.lemma_ for token in doc]
    lemma_text = ' '.join(lemma_tokens)
    return lemma_text

In [149]:
new_df['joined_tokens'] = new_df['joined_tokens'].astype('str')
new_df['joined_tokens'] = new_df['joined_tokens'].fillna('')
new_df['text-tfidf'] = new_df['joined_tokens'].apply(lemmatiz_text)


KeyboardInterrupt



In [68]:
# save in new_df
new_df.to_csv('created_new_train_data/new_11.csv')

In [63]:
new_df['Text'].iloc[250]

'I think we all do.'

In [42]:
# we need to clean the text first
print(new_df['cleaned_text'].iloc[3])

I must remind him the Democrats have controlled the Congress for the last twentytwo years and they wrote all the tax bills


In [7]:
new_df = new_df.dropna()

In [8]:
new_df.shape

(22385, 40)

In [9]:
text_data = new_df['text-tfidf']
labels = new_df['labels']

In [172]:
text_data

0        campaign cause question worry part leader acro...
1        let balance budget protect medicare medicaid e...
2                                        I d mention thing
3        must remind democrats control congress last tw...
4        take chance effort provide control weapon grea...
                               ...                        
22496                                     squander believe
22497                                           allow vote
22498    americans work history great percentage work f...
22499                        indicate intervene militarily
22500                    peace middle east nation interest
Name: text-tfidf, Length: 22385, dtype: object

## max features for tf-idf

In [74]:
# max_feature_range = [i for i in range(100, 55000, 200)]
# scores = {}
# for max_features in max_feature_range:
#     tfidf_vect = TfidfVectorizer(lowercase = True, analyzer='word', stop_words = 'english', ngram_range=(1, 2), max_features = max_features)
#     tfidf_features = tfidf_vect.fit_transform(text_data)

#     m = LogisticRegression()
#     # m = RandomForestClassifier()
#     cv_score = cross_val_score(m, tfidf_features, labels, cv = 5, scoring = 'accuracy').mean()

#     scores[max_features] = cv_score
#     print(f"Max Features: {max_features}, CV Accuracy: {cv_score: .4f}")

# best = max(scores, key = scores.get)
# print(f"Best Max Features: {best}, Accuracy: {scores[best]:.4f}")

## max features for wcb-tfidf

In [79]:
max_feature_range = [i for i in range(100, 5000, 100)]
scores = {}
for max_features in max_feature_range:
    wcbtfidf = Wcbtfidf(max_features = max_features)
    wcbtfidf_features = wcbtfidf.fit(text_data, labels)
    wcbtfidf_features_transformed = wcbtfidf.transform(text_data)

    m = LogisticRegression()
    # m = RandomForestClassifier()
    cv_score = cross_val_score(m, wcbtfidf_features_transformed, labels, cv = 5, scoring = 'accuracy').mean()

    scores[max_features] = cv_score
    print(f"Max Features: {max_features}, CV Accuracy: {cv_score: .4f}")

best = max(scores, key = scores.get)
print(f"Best Max Features: {best}, Accuracy: {scores[best]:.4f}")
# Max Features: 1500, CV Accuracy:  0.8108 

Max Features: 100, CV Accuracy:  0.7902
Max Features: 200, CV Accuracy:  0.7964
Max Features: 300, CV Accuracy:  0.8014
Max Features: 400, CV Accuracy:  0.8035
Max Features: 500, CV Accuracy:  0.8054
Max Features: 600, CV Accuracy:  0.8057
Max Features: 700, CV Accuracy:  0.8077
Max Features: 800, CV Accuracy:  0.8081
Max Features: 900, CV Accuracy:  0.8084
Max Features: 1000, CV Accuracy:  0.8099
Max Features: 1100, CV Accuracy:  0.8089
Max Features: 1200, CV Accuracy:  0.8090
Max Features: 1300, CV Accuracy:  0.8101
Max Features: 1400, CV Accuracy:  0.8105
Max Features: 1500, CV Accuracy:  0.8108
Max Features: 1600, CV Accuracy:  0.8107
Max Features: 1700, CV Accuracy:  0.8105
Max Features: 1800, CV Accuracy:  0.8097
Max Features: 1900, CV Accuracy:  0.8096
Max Features: 2000, CV Accuracy:  0.8096
Max Features: 2100, CV Accuracy:  0.8101
Max Features: 2200, CV Accuracy:  0.8090
Max Features: 2300, CV Accuracy:  0.8088
Max Features: 2400, CV Accuracy:  0.8089
Max Features: 2500, CV Ac

In [ ]:
# best_max_features = 4300
# weight_label_n = best_max_features * (17088/22501)
# weight_label_y = best_max_features * (5413/22501)
# print(weight_label_n)
# print(weight_label_y)

In [115]:
# max_feature_range
best = 10000
tfidf_vect = TfidfVectorizer(lowercase = True, analyzer='word', stop_words = 'english', ngram_range=(1, 2), max_features = best) # 10000
# joblib.dump(tfidf_vect, 'models/tfidf_vectorizer_10000.pkl')
tfidf_features = tfidf_vect.fit_transform(text_data)
joblib.dump(tfidf_vect, 'models/tfidf_vectorizer_10000.pkl')

['models/tfidf_vectorizer_10000.pkl']

## MODIFICATION 11.2
- make model of tfidf for 43000 features with traditional tfidf and wcbtfidf
- train the model test the model

In [73]:
from wcbtfidf import Wcbtfidf

In [13]:
# max_features_tfidf = 6300
# # 4300 - 81.03, 5300 - 81.05, 6300 - 81.06

# TFIDF
tfidf_vect = TfidfVectorizer(lowercase = True, analyzer='word', stop_words = 'english', ngram_range=(1, 2), max_features = 4300) # 10000
# joblib.dump(tfidf_vect, 'models/tfidf_vectorizer_10000.pkl')
tfidf_features = tfidf_vect.fit_transform(text_data)
joblib.dump(tfidf_vect, 'models/tfidf_vectorizer_4300.pkl')

# # WCBTFIDF
# max_features_wcbtfidf = 1500
# wcbtfidf = Wcbtfidf(max_features = max_features_wcbtfidf)
# wcbtfidf.fit(text_data, labels)
# wcb_tfidf_features = wcbtfidf.transform(text_data)
# # joblib.dump(wcbtfidf, 'models/wcb_tfidf_vectorizer_1500.pkl')

['models/tfidf_vectorizer_4300.pkl']

In [123]:
print(tfidf_features.shape)
print(tfidf_features.dtype)

(22501, 6300)
float64


In [174]:
print(wcb_tfidf_features.shape)
# print(wcb_tfidf_features.dtype)

(22385, 1499)


In [175]:
# we have convert it again for reducing the dimensionality -- then pca
new_df.rename(columns={'converted_embedding':'embedding'}, inplace = True)

In [176]:
additional_features = new_df[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [129]:
# additional_features.shape

In [177]:
embeddings = np.vstack(new_df['embedding'])
embeddings.shape

(22385, 1)

In [178]:
# we have convert it again for reducing the dimensionality -- then pca
# new_df.rename(columns={'converted_embedding':'embedding'}, inplace = True)
new_df['converted_embedding'] = new_df['embedding'].apply(convert_to_numpy_array)

### finding the est pca - start

### pca - end

In [179]:
pca = PCA(n_components = 50)
reduced_embeddings = pca.fit_transform(new_df['converted_embedding'].to_list())

In [90]:
# embeddings_np = np.array(reduced_embeddings)
# tfidf_features_np = tfidf_features.toarray()
# additional_features_np = np.array(additional_features)

In [180]:
embeddings_np = np.array(reduced_embeddings)
# wcb_tfidf_features_np = wcb_tfidf_features.toarray()
additional_features_np = np.array(additional_features)

In [181]:
X_train = np.hstack([wcb_tfidf_features,embeddings_np, additional_features_np])
y_train = new_df['labels']

## Random Forest (TFIDF features, embedding, additional features)

In [124]:
# pca = PCA(n_components = 500)
# X_reduced = pca.fit_transform(X)

In [92]:
best_params = {'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}

In [182]:
# X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

clf = RandomForestClassifier(**best_params)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=False, max_depth=90, min_samples_split=5,
                       n_estimators=800)

In [164]:
# print(X_train.shape)
# print(X_test.shape)

(16875, 10060)
(5626, 10060)


In [132]:
# y_pred = clf.predict(X_test)

In [183]:
# print(classification_report(y_test, y_pred))

In [ ]:
# base (checkworthy_2) precision    recall  f1-score   support

#            0       0.83      0.97      0.90      4229
#            1       0.84      0.41      0.55      1397

#     accuracy                           0.84      5626
#    macro avg       0.84      0.69      0.73      5626
# weighted avg       0.84      0.84      0.81      5626

## do the tfidf vectorization for test data

In [223]:
def joined_tokens(tokens):
    token_list = ast.literal_eval(tokens)
    return ' '.join(token_list)

In [186]:
# test_df = pd.read_csv('created_new_test_data/english_test_11_2.csv')

In [203]:
# test_df.drop(['text-tfidf'], axis = 1, inplace= True)
# test_df.drop(['joined_tokens'], axis = 1, inplace = True)
# test_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [230]:
test_df.dtypes

Sentence_id                  int64
Text                        object
Tokens                      object
text_length                  int64
cleaned_text                object
cleaned_text_length          int64
roberta_sentiment           object
roberta_sent_pos           float64
roberta_sent_neg           float64
roberta_sent_mixed         float64
roberta_sentiment_class     object
count_tokens                 int64
counts                      object
names                        int64
organizations                int64
dates                        int64
punctuation_count            int64
embedding                   object
converted_embedding         object
flattened_embedding         object
joined_tokens               object
text-tfidf                  object
dtype: object

In [231]:
# test_df['Tokens']

In [232]:
# # join the tokens
# test_df['joined_tokens'] = test_df['Tokens'].apply(joined_tokens)

# # lemmatize
# test_df['text-tfidf'] = test_df['joined_tokens'].apply(lemmatiz_text)

In [233]:
# test_df['text-tfidf'].isnull()
# test_df.to_csv('created_new_test_data/english_test_11_2.csv')

### NEW RFC: result is bad than previous model

In [137]:
print(clf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [136]:
# save random forest model
save_folder = 'models'
os.makedirs(save_folder, exist_ok = True)
model_path = os.path.join(save_folder, 'random_forest_tfidf_embeddings_10_features.pkl')
joblib.dump(clf, model_path)

['models\\random_forest_tfidf_embeddings_10_features.pkl']

## Logistic Regression (TFIDF features, embedding, additional features)

In [138]:
# lr = LogisticRegression(max_iter=1000)
# lr.fit(X_train,y_train)

C:\Users\looka\OneDrive\Documents\Thesis\master_thesis\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [139]:
y_pred = lr.predict(X_test)

In [140]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      4229
           1       0.78      0.62      0.69      1397

    accuracy                           0.86      5626
   macro avg       0.83      0.78      0.80      5626
weighted avg       0.86      0.86      0.86      5626



In [ ]:
# (base checkworthy_2)  Logistic Regression Results:
#               precision    recall  f1-score   support

#            0       0.87      0.94      0.91      4229
#            1       0.77      0.59      0.67      1397

#     accuracy                           0.85      5626
#    macro avg       0.82      0.76      0.79      5626
# weighted avg       0.85      0.85      0.85      5626


### NEW LR - result is also bad/similler

In [141]:
# save random forest model
save_folder = 'models'
os.makedirs(save_folder, exist_ok = True)
model_path = os.path.join(save_folder, 'lr_tfidf_embeddings_10_features.pkl')
joblib.dump(lr, model_path)

['models\\lr_tfidf_embeddings_10_features.pkl']

# try to test the gold test data with this new model

In [96]:
# load test dataset 
# test_df = pd.read_csv('created_new_test_data/gold_english_test_08_01.csv') # without label

In [99]:
## adding joined_tokens > lemmatize
# test_df['joined_tokens'] = test_df['Tokens'].apply(joined_tokens)
# test_df['text-tfidf'] = test_df['joined_tokens'].apply(lemmatiz_text)

In [104]:
# test_df['text-tfidf'] 

In [101]:
# test_df.to_csv('created_new_test_data/english_test_11_2.csv')

In [95]:
# same test data but with actual label
gold_df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/test_english_gold/CT24_checkworthy_english_test_gold.tsv',delimiter = '\t')

In [234]:
# test_df.dtypes

In [241]:
# tf idf
text = test_df['text-tfidf']
vectorizer_path = 'models/wcb_tfidf_vectorizer_1500.pkl'
wcb_tfidf_vect = joblib.load(vectorizer_path) ## loading the trained tfidf vectorizer
wcb_tfidf_features = wcb_tfidf_vect.transform(text)

In [59]:
# best = 10000
# tfidf_vect = TfidfVectorizer(lowercase = True, analyzer='word', stop_words = 'english', ngram_range=(1, 2), max_features = best) # 10000
# tfidf_features = tfidf_vect.fit_transform(text)

In [77]:
# test_df.drop(['converted_embedding', 'flattened_embedding'],axis = 1, inplace = True)

In [52]:
# new_df.rename(columns={'converted_embedding':'embedding'}, inplace = True)

In [237]:
additional_features = test_df[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [238]:
test_df['converted_embedding'] = test_df['embedding'].apply(convert_to_numpy_array)

In [239]:
pca = PCA(n_components = 50) # 356
reduced_embeddings = pca.fit_transform(test_df['converted_embedding'].to_list())

In [243]:
embeddings_np = np.array(reduced_embeddings)
# tfidf_features_np = wcb_tfidf_features.toarray()
additional_features_np = np.array(additional_features)

In [244]:
X = np.hstack([wcb_tfidf_features,embeddings_np, additional_features_np])
# y = test_df['labels']

In [129]:
print(tfidf_features_np.shape)  # Should be (num_samples, 10000)
print(embeddings_np.shape)      # Should be (num_samples, 50)
print(additional_features_np.shape)  # Should be (num_samples, 10)
print(X.shape)  # Should be (num_samples, 10060)

(341, 10000)
(341, 50)
(341, 10)
(341, 10060)


In [130]:
# load model lr
model_path_lr = 'models/lr_tfidf_embeddings_10_features.pkl'
model_lr = joblib.load(model_path_lr)
predictions_lr = model_lr.predict(X)
predictions_lr

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,

In [132]:
test_data_lr = pd.DataFrame({
    'id': test_df['Sentence_id'],
    'predictions': predictions_lr
})
test_data_lr['predictions'] = test_data_lr['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])

In [133]:
## for LR
df_merged_lr = pd.merge(gold_df, test_data_lr, left_on='Sentence_id', right_on='id', how='inner')


gold_labels = df_merged_lr['class_label'].tolist()
pred_labels = df_merged_lr['predictions'].tolist()

# Now use 'gold_labels' consistently
acc = accuracy_score(gold_labels, pred_labels)
precision = precision_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
recall = recall_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels, pred_labels, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7683284457478006
Precision: 0.5849056603773585
Recall: 0.3522727272727273
F1 Score: 0.4397163120567376


In [245]:
# load model rfc
# model_path_rfc = 'models/random_forest_tfidf_embeddings_10_features.pkl'
# model_rfc = joblib.load(model_path_rfc)
predictions_rfc = clf.predict(X)
# predictions_rfc

In [246]:
test_data_rfc = pd.DataFrame({
    'id': test_df['Sentence_id'],
    'predictions': predictions_rfc
})
test_data_rfc['predictions'] = test_data_rfc['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])

In [247]:
# for rfc
df_merged_rfc = pd.merge(gold_df, test_data_rfc, left_on='Sentence_id', right_on='id', how='inner')


gold_labels = df_merged_rfc['class_label'].tolist()
pred_labels = df_merged_rfc['predictions'].tolist()

# Now use 'gold_labels' consistently
acc = accuracy_score(gold_labels, pred_labels)
precision = precision_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
recall = recall_score(gold_labels, pred_labels, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels, pred_labels, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7478005865102639
Precision: 0.625
Recall: 0.056818181818181816
F1 Score: 0.10416666666666667


## OPTION 1 - Hyper parameter Tuning

### Using Scikit-Learn’s RandomizedSearchCV method, we can define a grid of hyperparameter ranges, 
### and randomly sample from the grid, performing K-Fold CV with each combination of values.

In [142]:
print(clf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [143]:
print(lr.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [147]:
rf = RandomForestRegressor(random_state = 42) ## look for the parameters

In [148]:
print('parameters:\n')
pprint(rf.get_params())

parameters:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [154]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] # number of trees in random forest
max_features = ['auto', 'sqrt'] # no. of features to consider at every split
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # max number of levels in tree
max_depth.append(None)
min_samples_split = [2,5,10]# minimum number of samples required to split node
min_samples_leaf = [1, 2, 4] # min number of samples required at each leaf node
bootstarp = [True, False] # method of selecting samples for training each tree

# create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstarp
}
pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


## from google colab

In [162]:
# # use the random grid to search for best hyperparameters
# # first create the base model to tune
# rf = RandomForestRegressor()
# # random search of parameters, using 3 fold cross validation
# # search across 100 different combinations, and use all availiable cores
# rf_random = RandomizedSearchCV(
#                             estimator = rf, 
#                             param_distributions = random_grid, 
#                             n_iter = 100, 
#                             cv = 3, 
#                             verbose = 2, 
#                             random_state = 42, 
#                             n_jobs = -1 # using all processors
#                             )
# # fit the random search model
# rf_random.fit(X_train, y_train)

#### evaluate random search

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error:{:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

    return accuracy

In [ ]:
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42) # base model
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
best_random = rf_random.best_estimator_ # best random search model
random_accuracy = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format(100 * (random_accuracy - base_accuracy)/ base_accuracy))

# resampling the imbalanced data

In [2]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)
x_train_o_balance, y_train_o_valance = smote.fit_resample(X_train, y_train)

In [ ]:
from imblearn.under_sampling import  RandomUnderSampler
rus = RandomUnderSampler(random_state = 42)
X_train_u_balanced, y_train_u_balanced = rus.fit_resample(X_train, y_train)